In [23]:
%%writefile "./py-files/three_strategy_with_loop.py"
import json
import numpy as np
import pandas as pd

bandit_state = [[1,1] for i in range(100)] 
total_reward = 0
last_step_bandit = 0
opponent_last = None
loop_current_step = 0
    
def bayesian_ts_bandit(observation, configuration):
    global bandit_state,total_reward, last_step_bandit, opponent_last, loop_current_step
    best_agent = None
    my_index = observation.agentIndex
    if my_index == 0:
        oppo_index = 1
    else:
        oppo_index = 0
    
    if observation.step%100 == 0:
        #print(f"Observations {observation}")
        print(f"3 step strategy with loop {total_reward}")
        
    if loop_current_step == 0:
        # initial bandit state
        bandit_state = [[1,1] for i in range(100)]
        
    if loop_current_step < 100:
        reward = observation.reward - total_reward
        total_reward += reward   
        
        # updating bandit_state using the result of the previous step       
        if reward > 0:
            # If winner increase the wins for that particular bandit
            bandit_state[last_step_bandit][0] += reward 
            best_agent = last_step_bandit
        else: 
            best_agent = loop_current_step
            # Increase the trials - wins for that particular bandit
            bandit_state[last_step_bandit][1] += 1 - reward

        if len(observation.lastActions) > 0:
            opponent_current = observation.lastActions[oppo_index]
            if opponent_last:
                if opponent_last == opponent_current:
                    bandit_state[opponent_current][0] += 1
            opponent_last = opponent_current  
    else:
        reward = observation.reward - total_reward
        total_reward += reward   
        
        # updating bandit_state using the result of the previous step       
        if reward > 0:
            # If winner increase the wins for that particular bandit
            bandit_state[last_step_bandit][0] += reward 
            best_agent = last_step_bandit
        else: 
            # Increase the trials - wins for that particular bandit
            bandit_state[last_step_bandit][1] += 1 - reward
        
        opponent_current = observation.lastActions[oppo_index]
        if opponent_last:
            if opponent_last == opponent_current:
                bandit_state[opponent_current][0] += 1
        opponent_last = opponent_current  

        best_proba = -1
        for k in range(100):
            proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
            if proba > best_proba:
                best_proba = proba
                best_agent = k

    last_step_bandit = best_agent
    loop_current_step += 1
    return best_agent

Overwriting ./py-files/three_strategy_with_loop.py
